In [1]:
import sys
import os
from pathlib import Path

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from compute_ET_blue import (
    compute_et_blue,
    compute_volumetric_et_blue,
    postprocess_et_blue,
)

from compute_ET_green import calculate_band_std_dev

from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset

from typing import List, Tuple, Union, Optional

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

### Postprocess and export ET Blue

In [5]:
TIME_STEPS = 12
TIME_STEP_TYPE = "monthly"
YEARS = range(2022, 2023)

In [6]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [7]:
ET_BLUE_RAW_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_landsat_10m_monthly_TG_2022"
).map(lambda img: back_to_float(img, 100))


date_utils.print_collection_dates(ET_BLUE_RAW_COLLECTION)
ee_utils.print_value_ranges(ET_BLUE_RAW_COLLECTION, "ET_blue")

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = -134.19, Max = 103.74
Image 2: Min = -24.75, Max = 35.90
Image 3: Min = -32.46, Max = 42.81
Image 4: Min = -55.35, Max = 44.48
Image 5: Min = -82.84, Max = 83.59
Image 6: Min = -103.69, Max = 86.83
Image 7: Min = -116.75, Max = 93.93
Image 8: Min = -81.88, Max = 73.52
Image 9: Min = -69.11, Max = 121.69
Image 10: Min = -116.12, Max = 714.27
Image 11: Min = 0.00, Max = 0.00
Image 12: Min = 0.00, Max = 0.00


In [8]:
ET_GREEN_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_green_landsat_10m_jurisdiction_monthly_TG_2022"
).map(lambda img: back_to_float(img, 100))

date_utils.print_collection_dates(ET_GREEN_COLLECTION)
ee_utils.print_value_ranges(ET_GREEN_COLLECTION, "ET_green")

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = 1.21, Max = 9.82
Image 2: Min = 10.53, Max = 32.27
Image 3: Min = 15.01, Max = 35.42
Image 4: Min = 39.07, Max = 64.96
Image 5: Min = 63.88, Max = 87.98
Image 6: Min = 81.91, Max = 117.42
Image 7: Min = 83.25, Max = 130.31
Image 8: Min = 55.71, Max = 104.72
Image 9: Min = 53.66, Max = 88.13
Image 10: Min = -0.30, Max = 42.72
Image 11: Min = 0.00, Max = 0.00
Image 12: Min = 0.00, Max = 0.00


In [9]:
def process_and_export_et_blue_postprocessed(
    et_blue_raw_list: ee.List,
    et_green_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 36,
    time_step_type: str = "dekadal",
) -> List[ee.batch.Task]:
    """
    Process and export post-processed ET blue images for a given year.

    Args:
        et_blue_raw_list (ee.List): List of raw ET blue images
        et_green_list (ee.List): List of ET green images
        year (int): Year to process
        year_index (int): Index of the year (e.g., 2022 would be 4 if starting from 2018)
        aoi (ee.Geometry): Area of interest to process
        time_steps (int): Number of time steps (default 36 for dekadal)
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        List[ee.batch.Task]: List of export tasks

    Raises:
        ValueError: If time_steps is not 12 or 36, or if time_step_type is invalid
    """
    if time_steps not in [12, 36]:
        raise ValueError("time_steps must be either 12 or 36")

    if time_step_type not in ["monthly", "dekadal"]:
        raise ValueError("time_step_type must be either 'monthly' or 'dekadal'")

    tasks = []
    et_blue_previous = None

    for i in range(time_steps):
        # Get current images
        et_green = ee.Image(et_green_list.get(i))
        et_blue_present = ee.Image(et_blue_raw_list.get(i))

        # Initialize previous for first iteration
        if et_blue_previous is None:
            et_blue_previous = et_blue_present

        # Process ET blue
        et_blue = _process_et_blue_image(
            et_blue_present=et_blue_present,
            et_blue_previous=et_blue_previous,
            et_green=et_green,
        )

        # Store current processed image for next iteration
        et_blue_previous = et_blue.select("ET_blue")

        # Generate time step name based on type
        time_step_name = _get_time_step_name(i, time_step_type)

        # Generate export task
        task = _create_export_task(
            et_blue=et_blue,
            year=year,
            time_step_name=time_step_name,
            time_step_type=time_step_type,
            aoi=aoi,
        )

        tasks.append(task)

    return tasks


def _process_et_blue_image(
    et_blue_present: ee.Image,
    et_blue_previous: ee.Image,
    et_green: ee.Image,
) -> ee.Image:
    """
    Process a single ET blue image with temporal post-processing.

    Args:
        et_blue_present (ee.Image): Current ET blue image
        et_blue_previous (ee.Image): Previous ET blue image
        et_green (ee.Image): Current ET green image

    Returns:
        ee.Image: Processed ET blue image with volumetric band
    """
    # Calculate threshold from ET green
    threshold = calculate_band_std_dev(et_green, "ET_green")

    # Post process using the previous processed image
    et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

    # Compute and add volumetric band
    et_blue_m3 = compute_volumetric_et_blue(et_blue)
    et_blue = et_blue.addBands(et_blue_m3)

    # Convert to int for storage
    return back_to_int(et_blue, 100)


def _get_time_step_name(index: int, time_step_type: str) -> str:
    """
    Generate the time step name based on index and type.

    Args:
        index (int): Current time step index
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        str: Formatted time step name
    """
    if time_step_type == "dekadal":
        month = index // 3 + 1
        dekad = index % 3 + 1
        return f"{month:02d}_D{dekad}"
    else:  # monthly
        month = index + 1
        return f"{month:02d}"


def _create_export_task(
    et_blue: ee.Image,
    year: int,
    time_step_name: str,
    time_step_type: str,
    aoi: ee.Geometry,
) -> ee.batch.Task:
    """
    Create an export task for a post-processed ET blue image.

    Args:
        et_blue (ee.Image): ET blue image to export
        year (int): Processing year
        time_step_name (str): Formatted time step name
        time_step_type (str): Type of time step
        aoi (ee.Geometry): Area of interest

    Returns:
        ee.batch.Task: Export task
    """
    task_name = f"ET_blue_postprocessed_landsat_10m_TG_{year}-{time_step_name}"

    asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_landsat_10m_monthly_TG_2022/{task_name}"

    return export_image_to_asset(
        image=et_blue,
        asset_id=asset_id,
        task_name=task_name,
        year=year,
        aoi=aoi,
        scale=10,
    )

In [10]:
all_tasks = []

for year in YEARS:

    et_blue_raw_collection_list = ET_BLUE_RAW_COLLECTION.filterDate(
        f"{year}-01-01", f"{year}-12-31"
    ).toList(TIME_STEPS)

    et_green_collection_list = ET_GREEN_COLLECTION.filterDate(
        f"{year}-01-01", f"{year}-12-31"
    ).toList(TIME_STEPS)

    tasks = process_and_export_et_blue_postprocessed(
        et_blue_raw_list=et_blue_raw_collection_list,
        et_green_list=et_green_collection_list,
        year=year,
        aoi=AOI,
        time_steps=TIME_STEPS,
        time_step_type=TIME_STEP_TYPE,
    )

    print(f"Year {year} processing complete. Started {len(tasks)} tasks.")

    all_tasks.extend(tasks)

print(f"Total tasks: {len(all_tasks)}")

Exporting ET_blue_postprocessed_landsat_10m_TG_2022-01 for 2022 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_landsat_10m_monthly_TG_2022/ET_blue_postprocessed_landsat_10m_TG_2022-01
Using projection EPSG:4326 at 10m resolution
Exporting ET_blue_postprocessed_landsat_10m_TG_2022-02 for 2022 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_landsat_10m_monthly_TG_2022/ET_blue_postprocessed_landsat_10m_TG_2022-02
Using projection EPSG:4326 at 10m resolution
Exporting ET_blue_postprocessed_landsat_10m_TG_2022-03 for 2022 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_landsat_10m_monthly_TG_2022/ET_blue_postprocessed_landsat_10m_TG_2022-03
Using projection EPSG:4326 at 10m resolution
Exporting ET_blue_postprocessed_landsat_10m_TG_2022-04 for 2022 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_landsat_10m_monthly_TG_2022/ET_blue_postprocessed_landsat_10m_TG_2022-04
Using projection EPSG:4326 at 10m resolution


### Sanity check

In [14]:
et_blue_postprocessed = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_landsat_10m_monthly_TG_2022"
).map(lambda img: back_to_float(img, 100))

# et_blue_postprocessed2 = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Zuerich/ET_blue_postprocessed_landsat_30m_monthly_ZH_2022"
# ).map(lambda img: back_to_float(img, 100))

date_utils.print_collection_dates(et_blue_postprocessed)
ee_utils.print_value_ranges(et_blue_postprocessed)

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = 0.00, Max = 103.73
Image 2: Min = -24.71, Max = 35.90
Image 3: Min = 0.00, Max = 42.81
Image 4: Min = 0.00, Max = 44.47
Image 5: Min = 0.00, Max = 83.58
Image 6: Min = 0.00, Max = 86.83
Image 7: Min = 0.00, Max = 93.92
Image 8: Min = 0.00, Max = 73.51
Image 9: Min = 0.00, Max = 121.69
Image 10: Min = -41.49, Max = 714.27
Image 11: Min = 0.00, Max = 0.00
Image 12: Min = 0.00, Max = 0.00


In [15]:
et_blue_postprocessed.first().bandNames().getInfo()

['ET_blue', 'ET_blue_m3']

In [16]:
Map = geemap.Map()

et_blue_postprocessed_list = et_blue_postprocessed.toList(12)
# et_blue_postprocessed2_list = et_blue_postprocessed2.toList(12)

image = ee.Image(et_blue_postprocessed_list.get(7))
# image2 = ee.Image(et_blue_postprocessed2_list.get(6))

vis_params = {
    "bands" : ["ET_blue"], 
    "min" : -10,
    "max" : 50,
    "palette" : ["white", "blue", "lightblue", "green", "yellow", "orange", "red"]
}

Map.center_object(AOI, 12)
Map.addLayer(image, vis_params, "ET blue postprocessed 10m")
# Map.addLayer(image2, vis_params, "ET blue postprocessed 30m")

Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…